# TODO:

* Move the code from 'new_models.py' to calendar_event and schedule_draft files.
* move the 
* make new helper functions for:
    - receive json generated by the timebox agent and convert to a ScheduleDraft
    - save the ScheduleDraft to the database
* make a new agent that receives the draft id and queries the database for that draft and then
  saves it to the calendar.

# Testing with a real schedule

# storing that draft

In [5]:
from typing import List
import json
from fateforger.agents.schedular.models import CalendarEvent
from fateforger.agents.schedular.models.schedule_draft import ScheduleDraft, DraftStore

events = [CalendarEvent.model_validate(event) for event in json.loads(timebox_json)]

event =  events[-1]
event.model_dump(exclude_none=True)


# gives str

{'calendarId': 'primary',
 'summary': 'Final prep',
 'start': datetime.datetime(2025, 7, 30, 22, 55),
 'end': datetime.datetime(2025, 7, 30, 23, 0),
 'timeZone': 'Europe/Amsterdam',
 'colorId': '3'}

# Committing that draft to google calendar

# New definitions

In [9]:
import pprint
from typing import List
from pydantic import TypeAdapter

schema: Dict[str, Any] = TypeAdapter(List[CalendarEvent]).json_schema(
    schema_generator=LLMJsonSchema,
    mode="validation"
)
pprint.pprint(schema)

{'$defs': {'Attendee': {'additionalProperties': False,
                        'properties': {'email': {'description': 'Email address '
                                                                'of the '
                                                                'attendee',
                                                 'format': 'email',
                                                 'title': 'Email',
                                                 'type': 'string'}},
                        'required': ['email'],
                        'title': 'Attendee',
                        'type': 'object'},
           'CalendarEvent': {'properties': {'attendees': {'anyOf': [{'items': {'$ref': '#/$defs/Attendee'},
                                                                     'type': 'array'},
                                                                    {'type': 'null'}],
                                                          'default': None,
                  

In [1]:
import json
timebox_json = '''
[
  {
    "event_type": "H",
    "summary": "Sleep",
    "start": "2025-08-02T03:00:00+02:00",
    "duration": "PT8H"
  },
  {
    "event_type": "H",
    "summary": "Morning routine",
    "duration": "PT1H"
  },
  {
    "event_type": "DW",
    "summary": "Deep Work – Partitional Dissonance",
    "duration": "PT2H"
  },
  {
    "event_type": "H",
    "summary": "Meditation",
    "duration": "PT10M"
  },
  {
    "event_type": "H",
    "summary": "Gym session",
    "duration": "PT1H45M"
  },
  {
    "event_type": "PR",
    "summary": "Personal planning session",
    "duration": "PT20M"
  },
  {
    "event_type": "R",
    "summary": "Lunch & buffer",
    "duration": "PT30M"
  },
  {
    "event_type": "SW",
    "summary": "Market trip",
    "duration": "PT1H30M"
  },
  {
    "event_type": "PR",
    "summary": "Planning session with girlfriend",
    "duration": "PT20M"
  },
  {
    "event_type": "BU",
    "summary": "Buffer before dinner",
    "duration": "PT25M"
  },
  {
    "event_type": "SW",
    "summary": "Cook dinner",
    "duration": "PT1H"
  },
  {
    "event_type": "BU",
    "summary": "Buffer before Baldur's Gate",
    "duration": "PT30M"
  },
  {
    "event_type": "R",
    "summary": "Play Baldur's Gate with girlfriend",
    "start": "2025-08-02T20:30:00+02:00",
    "duration": "PT1H30M"
  },
  {
    "event_type": "H",
    "summary": "Evening hygiene",
    "duration": "PT30M"
  },
  {
    "event_type": "SW",
    "summary": "Pack for next day",
    "duration": "PT30M"
  },
  {
    "event_type": "R",
    "summary": "Sci-fi reading",
    "duration": "PT1H",
    "end": "2025-08-03T00:00:00+02:00"
  },
  {
    "event_type": "H",
    "summary": "Sleep",
    "start": "2025-08-03T00:00:00+02:00",
    "duration": "PT8H"
  }
]
'''

In [ ]:
from fateforger.agents.schedular.models.new_models import CalendarEvent, ScheduleDraft, DraftStore
events = [CalendarEvent.model_validate(event) for event in json.loads(timebox_json)]
from datetime import datetime
today = datetime.now()


       # persists with filled start/end

In [ ]:
from datetime import date as Date
from sqlmodel import SQLModel, create_engine, Session, select
import json

# Clear SQLModel metadata
SQLModel.metadata.clear()

# Now import our models
# from fateforger.agents.schedular.models.calendar_event import CalendarEvent
# from fateforger.agents.schedular.models.schedule_draft import ScheduleDraft, DraftStore

from pydantic import TypeAdapter
from typing import List

draft  = ScheduleDraft(date=today, events=events)
draft.finalise()

# Create engine and tables
engine = create_engine("sqlite:///:memory:", echo=False)
SQLModel.metadata.create_all(engine)
with Session(engine) as session:
    store = DraftStore(session)
    saved_draft = store.save(draft) # TODO: make sure i get the id for querying later

# Query and save to calendar
The saver agent receives the draft id and queries the database for that draft and then saves it to the calendar.

In [ ]:
draft_store = DraftStore(session=session)  # type: ignore
draft_store.save(draft)   

In [3]:
import asyncio
import logging
from typing import Any, List

from tenacity import (
    retry,
    retry_any,
    retry_if_exception_type,
    retry_if_result,
    stop_after_attempt,
    wait_exponential,
    after_log,
)

class EventSubmitter:
    """
    Async batch submitter for MCP events, Python 3.10 edition:
      - per-call timeout via asyncio.wait_for()
      - exponential-backoff retries on exceptions or result.is_error
      - bounded parallelism (Semaphore)
      - gathers all tasks concurrently (asyncio.gather)
    """

    def __init__(
        self,
        mcp: Any,
        *,
        max_concurrency: int = 5,
        timeout: float = 10.0,
        retries: int = 5,
        base_delay: float = 0.5,
        max_delay: float = 5.0,
    ):
        self._mcp = mcp
        self._sem = asyncio.Semaphore(max_concurrency)
        self._timeout = timeout
        self._logger = logging.getLogger(self.__class__.__name__)
        self._retries = retries
        self._base_delay = base_delay
        self._max_delay = max_delay

        # retry on exception OR when .is_error is True
        retry_rule = retry_any(
            retry_if_exception_type(Exception),
            retry_if_result(lambda r: getattr(r, "is_error", False)),
        )

        async def _raw_rpc(event):
            # enforce per-call timeout in 3.10
            return await asyncio.wait_for(
                self._mcp.call_tool(
                    "create-event",
                    arguments=event.model_dump(exclude_none=True),
                ),
                timeout=self._timeout,
            )

        # wrap it in Tenacity’s retry decorator
        self._rpc = retry(
            retry=retry_rule,
            stop=stop_after_attempt(self._retries),
            wait=wait_exponential(multiplier=self._base_delay, max=self._max_delay),
            after=after_log(self._logger, logging.WARNING),
            reraise=True,
        )(_raw_rpc)

    async def _invoke_with_throttle(self, event) -> Any | None:
        """Acquire semaphore, call RPC (with retry+timeout), catch final failures."""
        async with self._sem:
            try:
                return await self._rpc(event)
            except Exception as exc:
                ev_id = getattr(event, "id", "<unknown>")
                self._logger.error("Event %s failed permanently: %s", ev_id, exc)
                return None  # keep slot in result list

    async def submit(self, draft) -> List[Any | None]:
        """
        Launch all create-event calls in parallel (up to max_concurrency).
        Returns a list of responses or None for each permanently failed event.
        """
        # fire off all workers
        tasks = [
            asyncio.create_task(self._invoke_with_throttle(ev))
            for ev in draft.events
        ]
        # wait for them all to finish, preserving order
        return await asyncio.gather(*tasks)


In [4]:
# from autogen_ext.tools.mcp import MCPWorkbench, StreamableHttpServerParams
from fateforger.tools.calendar_mcp import get_calendar_mcp_tools
import os
SERVER_URL = os.getenv("CALENDAR_MCP_URL", "http://localhost:3000")

from autogen_ext.tools.mcp import McpWorkbench
from autogen_ext.tools.mcp import StreamableHttpServerParams, mcp_server_tools



mcp_workbench = McpWorkbench(StreamableHttpServerParams(
        url=SERVER_URL,
        timeout=5,
    ))

In [5]:
# configure logging if you want to see warnings/errors
import logging
logging.basicConfig(level=logging.INFO)

submitter = EventSubmitter(
    mcp_workbench,
    max_concurrency=5,
    timeout=10.0,
    retries=5,
    base_delay=0.5,
    max_delay=5.0,
)

results = await submitter.submit(draft)
# results[i] is either the successful response or None if it failed after all retries


INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:mcp.client.streamable_http:Negotiated protocol version: 2025-03-26
INFO:mcp.client.streamable_http:Negotiated protocol version: 2025-03-26
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO